## folger pinnacle jul-aug 2021 hypoxic event

In [46]:
import pandas as pd
import onc
import os
import json
from typing import List, Tuple
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import helpers_1 as h1

from functools import reduce

#token = os.environ["TRICY_TOKEN"]
from dotenv import load_dotenv
load_dotenv()
token = os.getenv("ONC_TOKEN")

# Create ONC client
my_onc = onc.ONC(token)

Defining Hypoxia

"In ocean and freshwater environments, the term "hypoxia" refers to low or depleted oxygen in a water body." - https://oceanservice.noaa.gov/hazards/hypoxia/#:~:text=Graphic%20credit%3A%20Louisiana%20Universities%20Marine,oxygen%20in%20a%20water%20body.

"low levels of dissolved oxygen (less than 2-3 milligrams of oxygen per liter of water)" - https://www.epa.gov/ms-htf/hypoxia-101 



Effects of Hypoxia

- Deadzones
- 

Key Inidicators of Hypoxia

1. Oxygen Concentration and Saturation


2. Chloropyll
3. PAR (Photosynthetically Active Radiation)
4. Turbidity
5. Temperature
6. Salinity
7. Apparent Oxygen Utilization

In [47]:
"""
Parameters:
    wantedProps: list[str] - 
    locationCode: str - 

Returns:
    wantedSensorCategoryCodes: str - list to be directly passed to get_property function

"""

def find_properties_by_location(deviceCategoryCode: str, wantedProps: list[str], locationCode: str):
    
    wantedSensorCategoryCodes = "" # store result

    params = {
        "locationCode": locationCode,
        "deviceCategoryCode" : deviceCategoryCode
    }

    result = my_onc.getProperties(params)
    extracted = []

    for entry in result:
        # Defensive check: make sure these keys exist
        name = entry.get("propertyName", "")
        code = entry.get("propertyCode", "")
        has_data = entry.get("hasDeviceData", False)

        # Optionally: filter out properties that aren't actually measured
        if name and code:
            extracted.append({
                "propertyName": name,
                "propertyCode": code,
                "hasDeviceData": has_data
            })
    
    df = pd.DataFrame(extracted)
    df.head()

    for idx, row in df.iterrows(): # return index and series for each row
        
        for prop in wantedProps: # iterate through wanted properties
            if prop in row["propertyCode"]: # if wanted is a substring in the propertyCode of the current row
                wantedSensorCategoryCodes += f"{row["propertyCode"]},"
                wantedProps.remove(prop)

    # print properties the device cat at the location has and doesnt have
    print(f"{deviceCategoryCode} at {locationCode} does not have:")
    for prop in wantedProps:
        print(prop)
    print(f"{deviceCategoryCode} at {locationCode} does have: {wantedSensorCategoryCodes}")


In [48]:
#h1.find_properties_by_location("FGPPN")
wanted_properties = ["oxygen", "chlorophyll", "turbidity", "temperature", "salinity","parphotonbased"] #also want PAR
find_properties_by_location("FLNTU", wanted_properties, "FGPPN") # CTD doesnt have oxygne or chlorophyll


FLNTU at FGPPN does not have:
oxygen
temperature
salinity
parphotonbased
FLNTU at FGPPN does have: chlorophyll,turbidityntu,
